Import libraries and Load datasets

In [19]:
import pandas as pd
import glob
import os

# Load cleaned datasets from Phase 2

cleaned_folder = "cleaned_datasets"
files = glob.glob(os.path.join(cleaned_folder, "*.csv"))
print(f"Found {len(files)} files:", files)

students_list, courses_list, assessments_list = [], [], []

for file in files:
    df = pd.read_csv(file)
    dtype = df['Dataset_Type'].iloc[0]
    
    if dtype == "students":
        students_list.append(df)
    elif dtype == "courses":
        courses_list.append(df)
    elif dtype == "assessments":
        assessments_list.append(df)

# Merge datasets by type
students_df = pd.concat(students_list, ignore_index=True)
courses_df = pd.concat(courses_list, ignore_index=True)
assessments_df = pd.concat(assessments_list, ignore_index=True)

print("Datasets merged by type:")
print("Students:", students_df.shape)
print("Courses:", courses_df.shape)
print("Assessments:", assessments_df.shape)

Found 9 files: ['cleaned_datasets\\assessments_Huye_cleaned.csv', 'cleaned_datasets\\assessments_Kigali_cleaned.csv', 'cleaned_datasets\\assessments_Musanze_cleaned.csv', 'cleaned_datasets\\courses_Huye_cleaned.csv', 'cleaned_datasets\\courses_Kigali_cleaned.csv', 'cleaned_datasets\\courses_Musanze_cleaned.csv', 'cleaned_datasets\\students_Huye_cleaned.csv', 'cleaned_datasets\\students_Kigali_cleaned.csv', 'cleaned_datasets\\students_Musanze_cleaned.csv']
Datasets merged by type:
Students: (120, 11)
Courses: (12, 7)
Assessments: (143, 12)


<h1>1. Standardization: Z-score </h1>

In [20]:
# -----------------------------
# Scale numeric columns
# -----------------------------
numeric_cols = ['Mark', 'Attendance_Rate', 'Credits']

for col in numeric_cols:
    if col in assessments_df.columns:
        assessments_df[col + "_scaled"] = (assessments_df[col] - assessments_df[col].mean()) / assessments_df[col].std()

print("Numeric features scaled successfully!")

Numeric features scaled successfully!


<h1>2. Encoding </h1>

In [21]:
# -----------------------------
# One-hot encode Campus_Name, Program, Assessment_Type
# -----------------------------
onehot_cols = ['Campus_Name', 'Program', 'Assessment_Type']
existing_onehot_cols = [c for c in onehot_cols if c in assessments_df.columns]

assessments_df = pd.get_dummies(assessments_df, columns=existing_onehot_cols, drop_first=False)

# Convert all resulting one-hot columns to integers
onehot_new_cols = [c for c in assessments_df.columns if any(c.startswith(col+'_') for col in existing_onehot_cols)]
assessments_df[onehot_new_cols] = assessments_df[onehot_new_cols].astype(int)

print("One-hot encoding completed.")

One-hot encoding completed.


<h1> Binning </h1>

In [22]:
# -----------------------------
# Bin marks into performance bands
# -----------------------------
def performance_band(mark):
    if mark < 50:
        return "Fail"
    elif mark < 65:
        return "Pass"
    elif mark < 80:
        return "Credit"
    else:
        return "Distinction"

if 'Mark' in assessments_df.columns:
    assessments_df['Performance_Band'] = assessments_df['Mark'].apply(performance_band)

print("Marks binned into performance bands!")

# Optional preview
assessments_df['Performance_Band'].value_counts()

Marks binned into performance bands!


Performance_Band
Credit         70
Pass           60
Distinction     8
Fail            5
Name: count, dtype: int64

Clean Merge keys

In [23]:
# -----------------------------
# Clean merge keys
# -----------------------------
for df in [assessments_df, students_df, courses_df]:
    if 'Student_ID' in df.columns:
        df['Student_ID'] = df['Student_ID'].astype(str).str.strip().str.upper()
    if 'Campus_ID' in df.columns:
        df['Campus_ID'] = df['Campus_ID'].astype(str).str.strip()
    if 'Course_Code' in df.columns:
        df['Course_Code'] = df['Course_Code'].astype(str).str.strip().str.upper().str.replace("-", "")

# -----------------------------
# Handle duplicate students
# -----------------------------
students_df = students_df.sort_values('Intake_Year').drop_duplicates(subset=['Student_ID', 'Campus_ID'], keep='last').reset_index(drop=True)

# Resolve conflicting Full_Name and Program using mode
students_df['Full_Name'] = students_df.groupby(['Student_ID', 'Campus_ID'])['Full_Name'].transform(
    lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
)
students_df['Program'] = students_df.groupby(['Student_ID', 'Campus_ID'])['Program'].transform(
    lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
)

print("Merge keys cleaned and duplicate students resolved.")

# -----------------------------
# Merge students into assessments
# -----------------------------
silver_df = assessments_df.merge(
    students_df,
    on=['Student_ID', 'Campus_ID'],  # merge only on IDs
    how='left'
)

# Merge courses info
silver_df = silver_df.merge(
    courses_df,
    on=['Course_Code', 'Campus_ID'],
    how='left'
)

# Convert any remaining boolean columns to integers
for col in silver_df.select_dtypes('bool').columns:
    silver_df[col] = silver_df[col].astype(int)

print("Students and courses merged. Silver dataset ready.")
print("Silver dataset shape:", silver_df.shape)

Merge keys cleaned and duplicate students resolved.
Students and courses merged. Silver dataset ready.
Silver dataset shape: (143, 33)


Save transformed datasets

In [24]:
# -----------------------------
# Ensure transformed data is in silver_df
# -----------------------------
silver_df = assessments_df.copy()

print("Encoded + scaled columns now included in silver dataset.")
# Save silver dataset
# -----------------------------
silver_folder = "transformed_datasets"
os.makedirs(silver_folder, exist_ok=True)

silver_filepath = os.path.join(silver_folder, "silver_transformed.csv")
silver_df.to_csv(silver_filepath, index=False)

print(f"silver dataset saved: {silver_filepath}")

# Preview
silver_df.head(10)

Encoded + scaled columns now included in silver dataset.
silver dataset saved: transformed_datasets\silver_transformed.csv


,Student_ID,Course_Code,Mark,Assessment_Date,Academic_Year,Semester,Attendance_Rate,Source_Campus_File,Campus_ID,Dataset_Type,Mark_scaled,Attendance_Rate_scaled,Campus_Name_Huye,Campus_Name_Kigali,Campus_Name_Musanze,Assessment_Type_CAT,Assessment_Type_Formative,Assessment_Type_Summative,Performance_Band
0,HUY-26125,CS104,66.50,2023-06-16,2024/2025,1,0.8,Huyeassessmentscsv,HUY-26,assessments,0.058121,0.109766,1,0,0,0,1,0,Credit
1,HUY-26118,CS103,62.00,2023-01-08,2024/2025,2,0.6,Huyeassessmentscsv,HUY-26,assessments,-0.429581,-1.198281,1,0,0,0,0,1,Pass
2,HUY-26115,CS101,69.00,2023-01-08,2024/2025,1,0.6,Huyeassessmentscsv,HUY-26,assessments,0.329067,-1.198281,1,0,0,0,0,1,Credit
3,HUY-26107,CS102,66.50,2023-01-08,2024/2025,2,0.8,Huyeassessmentscsv,HUY-26,assessments,0.058121,0.109766,1,0,0,0,1,0,Credit
4,HUY-26105,CS101,58.00,2023-01-08,2024/2025,2,1.0,Huyeassessmentscsv,HUY-26,assessments,-0.863094,1.417813,1,0,0,0,0,1,Pass
5,HUY-26112,CS102,66.50,2023-01-08,2024/2025,2,1.0,Huyeassessmentscsv,HUY-26,assessments,0.058121,1.417813,1,0,0,0,1,0,Credit
6,HUY-26139,CS101,73.25,2023-01-08,2024/2025,1,0.8,Huyeassessmentscsv,HUY-26,assessments,0.789674,0.109766,1,0,0,1,0,0,Credit
7,HUY-26137,CS101,73.25,2023-01-08,2024/2025,1,0.8,Huyeassessmentscsv,HUY-26,assessments,0.789674,0.109766,1,0,0,1,0,0,Credit
8,HUY-26107,CS103,62.00,2023-06-21,2024/2025,1,0.8,Huyeassessmentscsv,HUY-26,assessments,-0.429581,0.109766,1,0,0,1,0,0,Pass
9,HUY-26103,CS104,66.50,2024-05-24,2024/2025,2,0.6,Huyeassessmentscsv,HUY-26,assessments,0.058121,-1.198281,1,0,0,0,1,0,Credit
